In [ ]:
import datetime
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from collections import namedtuple
from scipy import signal

from typing import List, Tuple

import gtsam
import numpy as np
from gtsam import ISAM2, Pose3, noiseModel, NavState
from gtsam.symbol_shorthand import B, V, X


import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

from common_setup import *

%matplotlib inline
# %matplotlib widget
import seaborn as sns
sns.set(font_scale=0.7)

# plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.max_open_warning'] = 50

dataset_folders = []

bagfile_folders = get_bagfile_folders(dataset_folders)
print("Found %d folders containing bag files:"%len(bagfile_folders), *bagfile_folders, sep='\n')

PDF_REPORT_OUT_FILE = 'reports/Custom_Report.pdf'
PDF_REPORT_RES = 120 # DPI

In [ ]:
# Register custom message definition
msg_definitions_folder = 'msg'
msg_types = load_msg_types(msg_definitions_folder)
register_types(msg_types)
print("\nRegistered %d ROS message definitions."%len(msg_types))

# Load Novatel Msg definition or any GNSS solution used
msg_definitions_folder = os.path.join('msg', "novatel_msgs")
msg_types = load_msg_types(msg_definitions_folder, 'novatel_oem7_msgs/msg/')
register_types(msg_types)
print("\nRegistered %d GNSS ROS message definitions."%len(msg_types))

In [ ]:
def load_bag(folder, topic="/TPCK_GNSS_STATE_MSG"):
    df = parse_bag(folder+'/bag', topic)
    bag_name = folder.split('/')[-1]

    print("Loaded %d datapoints from %s"%(len(df), bag_name ))
    
    # Filter data
    B_spd, A_spd = signal.butter(20, 1.0, 'low', fs=50)
    df['speed_lp'] = signal.filtfilt(B_spd, B_spd, df['abs_speed'])
    df['delta'] = df['ts'].diff()
    
    return(bag_name,df )

In [ ]:
bag_name,df = load_bag(bagfile_folders[0])
df


In [ ]:
GRAVITY = 0.0 #9.8

class NovatelCalibration:
    def __init__(self, body_ptx: float, body_pty: float, body_ptz: float,
                 body_prx: float, body_pry: float, body_prz: float,
                 accelerometer_sigma: float, gyroscope_sigma: float,
                 integration_sigma: float, 
                 accelerometer_bias_sigma: float,
                 gyroscope_bias_sigma: float):
        self.bodyTimu = Pose3(gtsam.Rot3.RzRyRx(body_prx, body_pry, body_prz),
                              gtsam.Point3(body_ptx, body_pty, body_ptz))
        self.accelerometer_sigma = accelerometer_sigma
        self.gyroscope_sigma = gyroscope_sigma
        self.integration_sigma = integration_sigma
        self.accelerometer_bias_sigma = accelerometer_bias_sigma
        self.gyroscope_bias_sigma = gyroscope_bias_sigma

calibration = NovatelCalibration( 0.0, 0.0, 0.0, 
                                  0.0, 0.0, 0.0, 
                                  0.01, 0.000175, 
                                  0.0, 
                                  0.0000000167, 
                                  2.91e-08 )


def getImuParams(calibration: NovatelCalibration):
    w_coriolis = np.zeros(3)

    # Set IMU preintegration parameters
    measured_acc_cov = np.eye(3) * np.power(
        calibration.accelerometer_sigma, 2)
    measured_omega_cov = np.eye(3) * np.power(
        calibration.gyroscope_sigma, 2)
    # error committed in integrating position from velocities
    integration_error_cov = np.eye(3) * np.power(
        calibration.integration_sigma, 2)

    imu_params = gtsam.PreintegrationParams.MakeSharedU(GRAVITY)
    # acc white noise in continuous
    imu_params.setAccelerometerCovariance(measured_acc_cov)
    # integration uncertainty continuous
    imu_params.setIntegrationCovariance(integration_error_cov)
    # gyro white noise in continuous
    imu_params.setGyroscopeCovariance(measured_omega_cov)
    imu_params.setOmegaCoriolis(w_coriolis)

    return imu_params

In [ ]:
gps_skip = 100

# Mark which GPS measurements to use
df['gps_measurement'] = 0
df['gps_measurement'].iloc[::gps_skip] = 1

for j,x in enumerate(['x','y','z']):
    df['pos_est_'+x] = np.NaN
    df['acc_bias_est_'+x] = np.NaN
    df['gyro_bias_est_'+x] = np.NaN
    df['vel_est_'+x] = np.NaN

# Configure noise models
noise_model_gps = noiseModel.Diagonal.Precisions(
    np.asarray([0, 0, 0] + [1.0 / 0.07] * 3))
    # np.asarray([1.0 / 0.08] * 3 + [1.0 / 0.07] * 3))

sigma_init_x = noiseModel.Diagonal.Precisions(
    np.asarray([1e-3, 1e-3, 1e-3, 1, 1, 1]))
sigma_init_v = noiseModel.Diagonal.Sigmas(np.ones(3) * 1000.0)
sigma_init_b = noiseModel.Diagonal.Sigmas(
    np.asarray([0.1] * 3 + [5.00e-05] * 3))

# Set ISAM2 parameters and create ISAM2 solver object
isam_params = gtsam.ISAM2Params()
isam_params.setFactorization("CHOLESKY")
isam_params.setRelinearizeSkip(10)

isam = gtsam.ISAM2(isam_params)

# Create the factor graph and values object that will store new factors and
# values to add to the incremental graph
new_factors = gtsam.NonlinearFactorGraph()
# values storing the initial estimates of new nodes in the factor graph
new_values = gtsam.Values()

# df is in time order
initialized = False

imu_params = getImuParams(calibration)
current_bias = gtsam.imuBias.ConstantBias()  # init with zero bias
current_imu = gtsam.PreintegratedImuMeasurements(imu_params, current_bias)
imu_count = 0
i = 0

for row in df.iloc[1800:10000].itertuples():
    # for row in df.itertuples():
    # print(row)
    
    # At each non=IMU measurement we initialize a new node in the graph
    # i = row.Index
    current_pose_key = X(i)
    current_vel_key = V(i)
    current_bias_key = B(i)
    t = row.ts

    if not initialized:
        if row.gps_measurement:
            # Create initial estimate and prior on initial pose, velocity, and biases
            current_pose_global = Pose3(gtsam.Rot3(*row.quat),row.pos)
            current_velocity_global = np.zeros(3)
            
            new_values.insert(current_pose_key, current_pose_global)
            new_values.insert(current_vel_key, current_velocity_global)
            new_values.insert(current_bias_key, current_bias)

            new_factors.addPriorPose3(current_pose_key, current_pose_global, sigma_init_x)
            new_factors.addPriorVector(current_vel_key, current_velocity_global, sigma_init_v)
            new_factors.addPriorConstantBias(current_bias_key, current_bias, sigma_init_b)
            
            i += 1
            t_previous = t
            initialized = True
        continue

    # We're initialized
    # Summarize IMU data between the previous GPS measurement and now
    current_imu.integrateMeasurement(row.acc,row.gyro,row.delta)
    imu_count += 1
            
    if row.gps_measurement:
        # Create IMU factor
        previous_pose_key = X(i-1)
        previous_vel_key = V(i-1)
        previous_bias_key = B(i-1)

        new_factors.push_back(
            gtsam.ImuFactor(previous_pose_key, previous_vel_key,
                            current_pose_key, current_vel_key,
                            previous_bias_key,
                            current_imu))
        current_imu = gtsam.PreintegratedImuMeasurements(imu_params, current_bias)
        imu_count = 0

        # Bias evolution as given in the IMU metadata
        sigma_between_b = gtsam.noiseModel.Diagonal.Sigmas(
            np.asarray([
                np.sqrt(imu_count) *
                calibration.accelerometer_bias_sigma
            ] * 3 + [
                np.sqrt(imu_count) *
                calibration.gyroscope_bias_sigma
            ] * 3))

        new_factors.push_back(
            gtsam.BetweenFactorConstantBias(previous_bias_key,
                                            current_bias_key,
                                            gtsam.imuBias.ConstantBias(),
                                            sigma_between_b))

        # Create GPS factor
        # gps_pose = Pose3(current_pose_global.rotation(),row.pos)
        gps_pose = Pose3(gtsam.Rot3(*row.quat),row.pos)
        
        new_factors.addPriorPose3(current_pose_key, gps_pose,
                                    noise_model_gps)
        # new_values.insert(current_pose_key, gps_pose)

        # Add initial values based on the previous estimates
        new_values.insert(current_pose_key, current_pose_global)
        new_values.insert(current_vel_key, current_velocity_global)
        new_values.insert(current_bias_key, current_bias)
        
        i += 1

        # Update solver
        if i > 2:
            if i == 3:
                print(f"############ NEW FACTORS AT TIME {t:.6f} ############")
                new_factors.print()
                print(f"############ NEW VALUES AT TIME {t:.6f} ############")
                new_values.print()

            isam.update(new_factors, new_values)

            # Reset the newFactors and newValues list
            new_factors.resize(0)
            new_values.clear()

            # Extract the result/current estimates
            result = isam.calculateEstimate()

            current_pose_global = result.atPose3(current_pose_key)
            current_velocity_global = result.atVector(current_vel_key)
            current_bias = result.atConstantBias(current_bias_key)

            if False:
                print(f"############ POSE AT TIME {t} ############")
                current_pose_global.print()
    
    nav_state_gps = NavState(current_pose_global,current_velocity_global)
    nav_state_dr = current_imu.predict(nav_state_gps,current_bias)
    for j,x in enumerate(['x','y','z']):        
        df['pos_est_'+x].iloc[row.Index] = nav_state_dr.pose().translation()[j]
        df['acc_bias_est_'+x].iloc[row.Index] = current_bias.accelerometer()[j]
        df['gyro_bias_est_'+x].iloc[row.Index] = current_bias.gyroscope()[j]
        df['vel_est_'+x].iloc[row.Index] = nav_state_dr.velocity()[j]
        
df

In [ ]:
# %matplotlib inline
%matplotlib widget

fig = plt.figure(figsize=(10,11))
fig.suptitle('')

gs = fig.add_gridspec(6,1)
# fig, axes = plt.subplots(nrows=4, ncols=4)

ax1 = [None] * 3
ax2 = [None] * 3

for i,x in enumerate(['x','y','z']):
    ax1[i] = fig.add_subplot(gs[2*i,:], sharex=ax1[0])

    if True:
        df.plot(kind='scatter', x='ts', y='pos_est_'+x, \
                 color='red', s=0.2, alpha=0.5, ax=ax1[i] )
        df.plot(kind='scatter', x='ts', y='pos_'+x, \
                color='blue', s=0.2, alpha=0.5, ax=ax1[i] )
    else:
        df.plot(kind='scatter', x='ts', y='vel_est_'+x, \
             color='red', s=0.2, alpha=0.5, ax=ax1[i] )
        df.plot(kind='scatter', x='ts', y='vel_'+x, \
                color='blue', s=0.2, alpha=0.5, ax=ax1[i] )


    ax2[i] = fig.add_subplot(gs[2*i+1,:], sharex=ax1[0])
    df.plot(kind='scatter', x='ts', y='acc_bias_est_'+x, \
            color='blue', s=0.2, alpha=0.5, ax=ax2[i] )

    df.plot(kind='scatter', x='ts', y='acc_'+x, \
            color='green', s=0.2, alpha=0.5, ax=ax2[i] )

fig.tight_layout()
